# 4.BodyTracking

### 카메라 인스턴스 생성

In [1]:
from jetbot import Camera
from jetbot import bgr8_to_jpeg

camera = Camera.instance(width=720, height=720)

### 카메라 Preview Widget 생성

In [2]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

detect_body = widgets.Image(width=300, height=300, format='jpeg')
display(detect_body)

Image(value=b'', format='jpeg', height='300', width='300')

### Cascade 모델 생성

In [3]:
# Haar-Cascade : fullbody 없으면 다운로드
!curl -L -O https://github.com/opencv/opencv/raw/master/data/haarcascades/haarcascade_fullbody.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  465k  100  465k    0     0   732k      0 --:--:-- --:--:-- --:--:-- 1428k


In [3]:
import cv2
body_cascade = cv2.CascadeClassifier('haarcascade_fullbody.xml')

# 검출할 객체 좌표 설정
body_x = body_y = body_w = body_h = 0

### PID 설정

In [4]:
import PID

xservo_pid = PID.PositionalPID(1.9, 0.3, 0.35)
yservo_pid = PID.PositionalPID(1.5, 0.2, 0.3)

### ServoMotor 설정

In [5]:
from servoserial import ServoSerial

servo = ServoSerial()

target_x = 2200
target_y = 2700

servo.Servo_serial_double_control(1, target_x, 2, target_y)

serial Open!
245
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\x98\x00\n\x02\n\x8c\x00\n\xf5'


WARNNIG: Jetson.GPIO library has not been verified with this carrier board,


### Camera -> boxing -> Body Tracking

In [6]:
while 1:
    frame = camera.value
    frame = cv2.resize(frame, (300, 300))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    bodies = body_cascade.detectMultiScale(gray)
    
    if len(bodies) > 0:
        (body_x, body_y, body_w, body_h) = bodies[0]
        
        cv2.rectangle(frame, (body_x, body_y), (body_x + body_w, body_y + body_h), (0, 255, 0), 4)
        
        # X, Y축 오프셋
        xservo_pid.SystemOutput = body_x + body_w / 2  # x축 중심점
        xservo_pid.SetStepSignal(150)
        xservo_pid.SetInertiaTime(0.01, 0.006)
        
        target_x = int(2200 + xservo_pid.SystemOutput)
        
        yservo_pid.SystemOutput = body_y + body_h / 2 # y축 중심점
        yservo_pid.SetStepSignal(150)
        yservo_pid.SetInertiaTime(0.01, 0.006)
        
        target_y = int(2700 + yservo_pid.SystemOutput)
        
        servo.Servo_serial_double_control(1, target_x, 2, target_y)
        
    detect_body.value = bgr8_to_jpeg(frame)    

229
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08~\x00\n\x02\n\xb6\x00\n\xe5'
224
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08o\x00\n\x02\n\xca\x00\n\xe0'
222
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08i\x00\n\x02\n\xd2\x00\n\xde'
189
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xa1\x00\n\x02\n\xbb\x00\n\xbd'
149
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xcf\x00\n\x02\n\xb5\x00\n\x95'
117
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xeb\x00\n\x02\n\xb9\x00\nu'
68
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\x0b\x00\n\x02\n\xc9\x00\nD'
125
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xcb\x00\n\x02\n\xd1\x00\n}'
152
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xbb\x00\n\x02\n\xc6\x00\n\x98'
147
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xc9\x00\n\x02\n\xbd\x00\n\x93'
146
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xd4\x00\n\x02\n\xb3\x00\n\x92'
148
b'\xff\xff\xfe\x0e\x83*\x04\x01\x08\xd6\x00\n\x02\n\xaf\x00\n\x94'
100
b'\xff\xff\xfe\x0e\x83*\x04\x01\t\x12\x00\n\x02\n\xa2\x00\nd'
50
b'\xff\xff\xfe\x0e\x83*\x04\x01\t:\x00\n\x02\n\xac\x00\n2'
233
b'\xff\xff\xfe\x0e\x83*\x04\x01\tZ\x0

KeyboardInterrupt: 